In [128]:
class Grille:

    def __init__(self,x_len,y_len):
        self.x_len = x_len  # Hauteur (commence en haut à 0) le nombre de liste
        self.y_len = y_len  # Largeur (commence à gauche à 0) le nombre d'élement dans chaque liste
        self.grille = self.create_grille()

    def create_grille(self)->list:
        
        grille=[]
        
        row = []
        for j in range(self.y_len):
            row.append("O")
        
        for i in range(self.x_len):
            grille.append(row.copy())
        
        return grille

    def update(self,liste_poisson,liste_requin):
        
        self.grille = self.create_grille()

        for poisson in liste_poisson:

            self.grille[poisson.x][poisson.y] = "P"

        for requin in liste_requin:
            self.grille[requin.x][requin.y] = "R"

    def __repr__ (self):

        affichage = ""
        for rangée in range(self.x_len):
            affichage += "\n" 
            for colonne in range(self.y_len):
                affichage += " " +self.grille[rangée][colonne]+ " "
        return affichage


            

class Poisson:

    nb_de_tour_pour_reproduction = 0

    def __init__(self,x,y):
        self.x = x
        self.y = y

        self.ancien_x = x
        self.ancien_y = y

        self.compteur = 0

    def se_reproduire(self,bebe_list,class_de_poisson):
        if self.compteur == self.nb_de_tour_pour_reproduction:
            # crée un enfant
            bebe = class_de_poisson(self.ancien_x, self.ancien_y)

            bebe_list.append(bebe)

            self.compteur = 0



    def coord_voisins(self,grille):
        if self.x !=0:
            up = (self.x-1,self.y)
        else:
            up = (grille.x_len-1,self.y)


        if self.x !=grille.x_len-1:
            down = (self.x+1,self.y)
        else:
            down = (0,self.y)

        if self.y !=0:
            left = (self.x,self.y-1)
        else:
            left = (self.x,grille.y_len-1)

        if self.y !=grille.y_len-1:
            right = (self.x,self.y+1)
        else:
            right = (self.x,0)

        dictionnaire_coord_voisin = {
            "up":up,
            "down":down,
            "right":right,
            "left":left
        }
        return dictionnaire_coord_voisin

    def move(self,grille,bebe_poisson_list):

            # un tour à lieu pour le requin
            self.compteur+=1

            # Je recherche les coordonées des voisins
            dict_coord_voisin = self.coord_voisins(grille)


            # Trier les coordonnées des voisins en fonction de poisson et vide
            liste_des_cases_autour_vides = []


            for i , j in dict_coord_voisin.values():
                if grille.grille[i][j] == "O":
                    liste_des_cases_autour_vides.append((i,j))

            if len(liste_des_cases_autour_vides)>0:

                # choisi un des voisins poisson comme cible
                cible = rd.choice(liste_des_cases_autour_vides)
                # fait bouger le requin vers cette cible
                self.ancien_x = self.x
                self.ancien_y = self.y
                
                self.x = cible[0]
                self.y = cible[1] 
                    
                
                # il se reproduit
                self.se_reproduire(bebe_poisson_list,Poisson)







class Requin(Poisson):

    energy_par_poisson = 0
    start_energy = 0
    
    def __init__(self,x,y):
        super().__init__(x,y)

        self.energy = self.start_energy
        


    def move(self,grille:Grille,une_liste_de_poissons,bebe_requin_list,une_liste_de_requins):

        # un tour à lieu pour le requin
        self.compteur+=1

        # Je recherche les coordonées des voisins
        dict_coord_voisin = self.coord_voisins(grille)


        # Trier les coordonnées des voisins en fonction de poisson et vide
        liste_des_cases_autour_avec_du_poisson = []
        liste_des_cases_autour_vides = []


        for i , j in dict_coord_voisin.values():
            if grille.grille[i][j] == "P":
                liste_des_cases_autour_avec_du_poisson.append((i,j))
            elif grille.grille[i][j] == "O":
                liste_des_cases_autour_vides.append((i,j))

        ########## si l'un des voisins c'est du poissons
        if len(liste_des_cases_autour_avec_du_poisson)>0:
            # choisi un des voisins poisson comme cible
            cible = rd.choice(liste_des_cases_autour_avec_du_poisson)
            # fait bouger le requin vers cette cible
            self.ancien_x = self.x
            self.ancien_y = self.y
            
            self.x = cible[0]
            self.y = cible[1] 

            # ajouter et enlever de l'energie au requin
            self.energy += self.energy_par_poisson
            self.energy -= 1

            # faire disparaitre le poisson mangé de la liste de poissons
            for poisson in une_liste_de_poissons:
                if poisson.x == cible[0] and poisson.y == cible[1]:
                    une_liste_de_poissons.remove(poisson)

            self.se_reproduire(bebe_requin_list,Requin)

            
        elif len(liste_des_cases_autour_vides)>0:

            # choisi un des voisins poisson comme cible
            cible = rd.choice(liste_des_cases_autour_vides)
            # fait bouger le requin vers cette cible
            self.ancien_x = self.x
            self.ancien_y = self.y
            
            self.x = cible[0]
            self.y = cible[1] 

            #perd de lénergy
            self.energy -=1
            
            # il meurt
            if self.energy == 0:
                une_liste_de_requins.remove(self)
                
            
            # il se reproduit
            self.se_reproduire(bebe_requin_list,Requin)


        else:
            self.energy -=1
            # il meurt
            if self.energy == 0:
                une_liste_de_requins.remove(self)



            
import random as rd



def create_animal_lists(x_len,y_len,nb_poisson,nb_requin):
    mon_set=set()
    while len(mon_set) < nb_poisson + nb_requin:
        x = rd.randint(0,x_len-1)
        y = rd.randint(0,y_len-1)
        coord = (x,y)
        mon_set.add(coord)


    liste_poisson = []
    liste_requin = []



    for coord in list(mon_set)[0:nb_poisson]:
        liste_poisson.append( Poisson(coord[0],coord[1]))

    for coord in list(mon_set)[nb_poisson :  ]:
        liste_requin.append( Requin(coord[0],coord[1]))

    return liste_poisson, liste_requin


# print(len(list(mon_set)[nb_poisson:len(mon_set)]))

In [129]:

######### INITIALISATION

Requin.nb_de_tour_pour_reproduction = 15
Requin.energy_par_poisson = 2
Requin.start_energy = 4
Poisson.nb_de_tour_pour_reproduction = 2

x_len_test = 5
y_len_test = 8

ma_grille = Grille(x_len_test,y_len_test)

une_liste_de_poissons,une_liste_de_requins = create_animal_lists( x_len=x_len_test, y_len=y_len_test, nb_poisson=7, nb_requin=2)

ma_grille.update(une_liste_de_poissons,une_liste_de_requins)

print("----Initialisation -----")
print("nb_requin",len(une_liste_de_requins))
print("nb_poisson",len(une_liste_de_poissons))
print(ma_grille)


###########LES TOURS
jeu_non_termine = True

tour = 0
while jeu_non_termine:
    bebe_requin_list=[]

    for requin in une_liste_de_requins:
        requin.move(ma_grille,une_liste_de_poissons,bebe_requin_list,une_liste_de_requins)
        jonction_requin = []
        jonction_requin.extend(une_liste_de_requins)
        jonction_requin.extend(bebe_requin_list) 
        ma_grille.update(une_liste_de_poissons,jonction_requin)
        # print("----Requin suivant -----")
        # print("nb_requin",len(jonction_requin))
        # print("nb_poisson",len(une_liste_de_poissons))
        # print(ma_grille)

    une_liste_de_requins.extend(bebe_requin_list)


    bebe_poisson_list=[]
    for poisson in une_liste_de_poissons:  
        poisson.move(ma_grille,bebe_poisson_list)
        jonction_poisson = []
        jonction_poisson.extend(une_liste_de_poissons)
        jonction_poisson.extend(bebe_poisson_list) 
        ma_grille.update(jonction_poisson,une_liste_de_requins)
        # print("----POisson suivant -----")
        # print("nb_requin",len(une_liste_de_requins))
        # print("nb_poisson",len(jonction_poisson))
        # print(ma_grille)


    une_liste_de_poissons.extend(bebe_poisson_list)

    print("----TOur n ----- ", tour)
    print("nb_requin",len(une_liste_de_requins))
    print("nb_poisson",len(une_liste_de_poissons))
    print(ma_grille)


    tour +=1
    ### Analyse de la fin de jeu:

    if len(une_liste_de_poissons) == 0:
        print("TOus les poissons sont morts, le jeu est fini")
        print (" tour ", tour)
        jeu_non_termine = False
    elif len(une_liste_de_requins) == 0:
        print("tous les requins sont morts")
        print (" tour ", tour)
        jeu_non_termine = False
    elif tour == 150:
        print("Vous avez attient le tour 150, cest un bel équilibre")
        jeu_non_termine = False

----Initialisation -----
nb_requin 2
nb_poisson 7

 O  O  O  O  O  P  O  O 
 O  O  O  O  O  O  O  O 
 P  P  P  O  O  R  O  O 
 P  O  O  O  O  O  O  O 
 O  O  P  O  P  O  O  R 
----TOur n -----  0
nb_requin 2
nb_poisson 7

 O  O  O  O  P  O  O  R 
 O  O  O  O  O  R  O  O 
 O  O  O  P  O  O  O  P 
 O  P  O  O  O  O  O  O 
 P  O  O  P  O  P  O  O 
----TOur n -----  1
nb_requin 2
nb_poisson 14

 R  O  O  P  P  R  O  O 
 O  O  O  O  O  O  O  O 
 O  P  O  P  O  O  O  P 
 P  P  O  P  O  P  O  P 
 P  O  O  P  P  P  O  O 
----TOur n -----  2
nb_requin 2
nb_poisson 12

 O  O  P  O  O  O  O  O 
 O  P  O  P  O  O  O  P 
 O  P  O  O  O  P  O  O 
 O  O  P  P  P  O  P  P 
 R  O  O  O  P  R  O  O 
----TOur n -----  3
nb_requin 2
nb_poisson 22

 O  P  P  P  O  O  O  O 
 O  P  O  P  O  O  O  P 
 P  P  P  P  P  P  P  P 
 O  O  P  P  P  O  P  P 
 O  R  O  P  R  O  P  P 
----TOur n -----  4
nb_requin 2
nb_poisson 20

 O  R  O  P  P  O  O  P 
 P  P  P  O  P  O  P  O 
 O  O  P  P  P  O  P  O 
 P  P  P  O  R 

In [104]:
print(ma_grille)


 O  O  O  O  O  O  P  O 
 O  O  O  O  O  P  O  R 
 O  P  O  O  O  O  O  O 
 O  O  O  O  P  O  O  R 
 R  O  O  O  O  O  O  O 


In [105]:
print(Requin(3,6))

In [106]:
print(list([3,4,5]))

[3, 4, 5]


In [124]:
class Voiture:

    def __init__(self,marque, couleur):
        self.marque = marque
        self.couleur = couleur

    def clone(self):
        return self.__class__.

print(Voiture("renaud","bleu").clone())

None
